In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
class Scenario(QualitativeScenario):
    def __init__(self):
        QualitativeScenario.__init__(self, "bla")

In [ ]:
class ActivityClass:
    def __init__(self, name, model, state):
        self.name = name
        self.model = model
        self.state = state

In [ ]:
class Activity(ActivityClass):
    def __init__(self, name, model, state, tstart, tend, parameters):
        ActivityClass.__init__(self, name, model, state)
        self.tstart = tstart
        self.tend = tend
        self.tduration = tend - tstart
        self.parameters = parameters
        
        # Set model parameters
        self.model.set_parameters(self.parameters)
        
    def plot_state(self):
        x = np.linspace(self.tstart, self.tend, 100)
        y = self.model.get_state(n=len(x))
        plt.plot(x, y)
        plt.xlabel("Time")
        plt.ylabel(self.state)
        
    def plot_state_dot(self):
        x = np.linspace(self.tstart, self.tend, 100)
        y = self.model.get_state_dot(n=len(x)) / (self.tend - self.tstart)
        plt.plot(x, y)
        plt.xlabel("Time")
        plt.ylabel("{:s} dot".format(self.state))

In [ ]:
class Model:
    def __init__(self):
        self.p = {}
        
    def set_parameters(self, parameters):
        self.p = parameters
    
class spline_3_knots(Model):
    def get_state(self, n=100):
        x = np.linspace(0, 1, n)
        x1 = x[:n//2]
        x2 = x[n//2:]
        y1 = self.p["a1"]*x1**3 + self.p["b1"]*x1**2 + self.p["c1"]*x1 + self.p["d1"]
        y2 = self.p["a2"]*x2**3 + self.p["b2"]*x2**2 + self.p["c2"]*x2 + self.p["d2"]
        y = np.concatenate((y1, y2))
        return self.p["xstart"] + y*(self.p["xend"] - self.p["xstart"])
    
    def get_state_dot(self, n=100):
        x = np.linspace(0, 1, n)
        x1 = x[:n//2]
        x2 = x[n//2:]
        y1 = 3*self.p["a1"]*x1**2 + 2*self.p["b1"]*x1 + self.p["c1"]
        y2 = 3*self.p["a2"]*x2**2 + 2*self.p["b2"]*x2 + self.p["c2"]
        y = np.concatenate((y1, y2))
        return y*(self.p["xend"] - self.p["xstart"])

class linear(Model):
    def get_state(self, n=100):
        return np.linspace(self.p["xstart"], self.p["xend"], n)
    
    def get_state_dot(self, n=100):
        return np.ones(n) * (self.p["xend"] - self.p["xstart"])

In [ ]:
acc = ActivyClass("acceleration", spline_3_knots(), "x")
ego_acceleration1 = Activity(acc, 0, 8.97, 
                            {"xstart": 0, "xend": 133, "a1": -7.55e-2, "b1": 3.62e-1, "c1": 7.07e-1, "d1": 0,
                             "a2": -2.18e-2, "b2": 2.82e-1, "c2": 7.47e-1, "d2": -6.73e-3})
ego_braking = Activity("braking", spline_3_knots(), "x", 9.00, 10.98,
                       {"xstart": 133, "xend": 168, "a1": 1.56e-2, "b1": -6.27e-2, "c1": 1.04, "d1": 0,
                        "a2":3.31e-2, "b2": -8.89e-2, "c2": 1.06, "d2": -2.18e-3})
ego_cruising1 = Activity("cruising", linear(), "x", 11, 17.98, {"xstart": 168, "xend": 287})
ego_acceleration2 = Activity("acceleration", spline_3_knots(), "x", 18, 22.98,
                             {"xstart": 288, "xend": 381, "a1": -6.8e-2, "b1": 1.45e-1, "c1": 9.17e-1, "d1": 0,
                              "a2": -3.12e-2, "b2": 9.09e-2, "c2": 9.45e-1, "d2": -4.6e-3})
ego_cruising2 = Activity("cruising", linear(), "x", 23, 42.3, {"xstart": 381, "xend": 756})

In [ ]:
ego_acceleration1.plot_state()
ego_braking.plot_state()
ego_cruising1.plot_state()
ego_acceleration2.plot_state()
ego_cruising2.plot_state()
plt.show()

In [ ]:
ego_acceleration1.plot_state_dot()
ego_braking.plot_state_dot()
ego_cruising1.plot_state_dot()
ego_acceleration2.plot_state_dot()
ego_cruising2.plot_state_dot()
plt.show()

In [ ]:
class Actor:
    def __init__(self, name, list_of_activity):
        self.name = name
        self.list_of_activity = list_of_activity
        
    def plot_state(self, state):
        for activity in self.activities:
            if activity.state == state:
                activity.plot_state()
                
    def plot_state_dot(self, state):
        for activity in self.activities:
            if activity.state == state:
                activity.plot_state_dot()

In [ ]:
ego = Actor("ego", [ego_acceleration1, ego_braking, ego_cruising1])
ego.plot_state_dot("x")
plt.show()

In [ ]:
pickuptruck = Actor("actor", [Activity("cruising", linear(), "x", 0.8, 33.4, {"xstart": 117, "xend": 556})])
pickuptruck.plot_state_dot("x")
plt.show()

In [ ]:
class Scenario:
    def __init__(self, actors):
        self.actors = actors
    
    def plot_state(self, state):
        for actor in self.actors:
            actor.plot_state("x")

In [ ]:
scenario1 = Scenario([ego, pickuptruck])
scenario1.plot_state("x")